In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [10]:
import warnings
warnings.filterwarnings("ignore")

# OpenAI Assistants API를 활용한 서비스 구현 튜토리얼 (2025년 3월 기준)

본 튜토리얼에서는 OpenAI Assistants API를 활용하여 AI 어시스턴트 기반 서비스를 개발하는 방법을 단계별로 설명합니다. 2025년 3월 기준의 OpenAI 공식 API 문서를 기반으로 최신 기능과 모범 사례를 반영했으며, Python 예제 코드를 통해 실습 형태로 진행됩니다. 이 튜토리얼은 ChatGPT 등의 언어 모델 API 사용 경험이 있고, 새로운 Assistants API 기능에 관심이 있는 개발자를 대상으로 합니다.

## 1. OpenAI Assistants API 소개

OpenAI Assistants API는 OpenAI의 새로운 API로, 대화형 AI 어시스턴트를 보다 쉽게 구축하고 관리할 수 있도록 설계되었습니다. 기존의 Chat Completion API가 무상태(stateless) 방식이어서 매 요청마다 대화 기록을 모두 보내야 했다면, Assistants API는 상태를 유지(stateful) 하는 대화 스레드(Thread)를 제공하여 자동으로 대화 컨텍스트를 관리합니다. 또한 함수 호출, 코드 실행, 지식 검색 등의 도구(Tools) 를 어시스턴트에 병렬로 연결할 수 있어 훨씬 강력한 코파일럿(copilot) 형태의 서비스를 구축할 수 있습니다. 

왜 Assistants API를 사용해야 할까요? 몇 가지 주요 장점을 정리하면 다음과 같습니다:

- 대화 지속성 & 메모리: Assistants API는 대화 스레드에 메시지 내역을 지속적으로 저장하며, 컨텍스트 윈도우 제한 내에서 자동으로 요약/잘라내기를 수행합니다. 개발자는 별도의 상태 관리 로직 없이 장기간의 대화도 처리할 수 있습니다.

- 다양한 도구 통합: 어시스턴트가 코드 실행(Code Interpreter), 지식 검색(File Retrieval), 함수 호출(Function Calling) 등의 도구를 직접 사용하도록 구성할 수 있습니다. 이를 통해 모델의 한계를 넘어서는 작업(예: 데이터베이스 질의, 파일 처리, 외부 API 호출 등)을 자동화할 수 있습니다.

- 개인화된 지시어: 어시스턴트를 생성할 때 지시어(Instructions) 를 주입함으로써 해당 어시스턴트의 성격과 역할을 미리 정의할 수 있습니다. 예를 들어 “당신은 친절한 고객지원 봇입니다”와 같은 시스템 지시어로 어시스턴트의 톤과 도메인 지식을 설정할 수 있습니다.

- 모델 기능 향상: 2023년 11월 이후 공개된 GPT-3.5/4 모델 (예: gpt-3.5-turbo-1106, gpt-4-1106-preview)은 함수 호출과 같은 고급 기능을 지원하며 Assistants API와 연동됩니다. Assistants API를 사용하면 이러한 최신 모델 기능을 간편하게 활용할 수 있습니다.

- 개발 편의: OpenAI 플랫폼의 웹 인터페이스(플레이그라운드)뿐 아니라 API를 통해서도 동일한 어시스턴트를 생성/관리할 수 있습니다. 한 번 만든 어시스턴트를 재사용하거나 여러 쓰레드를 동시에 운영하는 것이 쉬워집니다. (예: 하나의 어시스턴트에 대해 다수의 사용자 대화 스레드를 관리)

요약하면, Assistants API는 기존 ChatGPT API의 상태 관리 진화판으로 볼 수 있습니다. 복잡한 대화 상태 관리, 외부 도구 통합, 문서 임베딩 등 많은 부분을 OpenAI 플랫폼이 맡아주므로, 개발자는 핵심 로직 구현에 집중할 수 있습니다. 이러한 이유로, 코파일럿 스타일의 챗봇이나 자동화 에이전트를 만든다면 Assistants API가 강력한 선택지가 됩니다.

>Note: Assistants API는 2025년 초 현재 베타(beta) 단계로, OpenAI가 지속적으로 기능을 개선하고 있습니다. 사용 전에 최신 문서를 확인하고, 베타 API인 만큼 일부 동작이나 요금 정책이 변경될 수 있음을 유의하세요.

## 2. API 키 설정 및 환경 변수 사용

OpenAI API를 사용하려면 API 키가 필요합니다. OpenAI 플랫폼의 API 키 관리 페이지에서 비밀 키를 생성할 수 있습니다. 생성된 키는 한 번만 표시되므로, 반드시 복사하여 안전한 곳에 저장하세요. 일반적으로 이 키를 소스 코드에 하드코딩하지 않고, 별도의 설정으로 관리하는 것이 좋습니다. **환경 변수(Environment Variable)**를 사용하면 API 키를 소스 코드에 노출하지 않고 관리할 수 있습니다. 개발 PC 또는 서버의 환경 변수 OPENAI_API_KEY에 키를 저장해 두면, OpenAI 라이브러리가 자동으로 이를 읽어 사용할 수 있습니다. Python 개발 환경에서는 python-dotenv 패키지를 활용해 .env 파일에 키를 저장하고 로드하는 방식이 편리합니다. 

다음은 API 키를 설정하고 로드하는 과정입니다:

1. python-dotenv 설치: 터미널에서 pip install python-dotenv 명령으로 설치합니다 (한번만 수행).

2. 환경 변수 파일 생성: 프로젝트 루트 디렉토리에 .env 파일을 만들고, 아래와 같이 OpenAI API 키를 입력합니다 (따옴표 없이 실제 키로 대체).

    ```
    OPENAI_API_KEY=sk-***********************
    ```
    
3. 코드에서 로드: Python 코드에서 python-dotenv를 이용해 .env를 로드하고, os.environ을 통해 키를 불러옵니다. OpenAI 공식 Python SDK에서는 환경 변수 OPENAI_API_KEY를 자동으로 인식하므로, 명시적으로 지정하지 않아도 동작합니다. 그래도 명확성을 위해 아래와 같이 작성할 수 있습니다:


In [3]:
from dotenv import load_dotenv
from decouple import config
from openai import OpenAI
import os
# 1. client 생성
load_dotenv(".env") # 기본값 ".env"
# api_key = os.getenv('OPENAI_API_KEY')
# api_key = config('OPENAI_API_KEY')
# print(api_key[:5])
client = OpenAI()

In [ ]:
# 단순요청
# response = client.chat.completions.create(
#     model = '',
#     # messages=[{"role":"user", "content":"서울 날씨가 어떤가요?"}]
#     messages="오늘 날씨가 어떤가요?"
# )
# Assistant API이용하여 (1) 대화이력을 저장 (2)파일 첨부 (3) 함수호출(ex.날씨를 반환하는 함수)

## 3. 기본적인 Assistant 생성 및 메시지 보내기

가장 먼저, **어시스턴트(Assistant)**를 하나 만들어보겠습니다. 어시스턴트는 대화를 수행하는 주체로, 사용자의 요청을 받아 처리하는 AI 에이전트라고 볼 수 있습니다. Assistants API에서 어시스턴트를 생성할 때는 어떤 모델을 사용할지, 어떤 지시어(Instructions) 를 가질지, 그리고 사용할 **툴(Tools)**이 무엇인지 등을 설정할 수 있습니다. 예를 들어 간단한 챗봇 어시스턴트를 만들어 사용자 질문에 답변해보겠습니다.


In [7]:
# 2. assistant 생성 : 인공지능 캐렉터(기능 정의)
assistant = client.beta.assistants.create(
    name="HelpBot",
    instructions="당신은 유능하고 친절한 도움말 어시스턴트입니다. 사용자 질문에 20자이내로 답변하세요",
    model="gpt-4o-mini",
    # tools=[]
)

In [8]:
assistant
print('Assistant ID :', assistant.id)

Assistant ID : asst_4gBqxGrrNXDNLQiMg9XWT175


위 코드에서는 client.beta.assistants.create 메서드를 사용해 새로운 어시스턴트를 생성했습니다. 주요 파라미터를 살펴보면:

- name: 어시스턴트의 이름을 지정합니다. 콘솔이나 리스트에서 구분하기 위한 용도이며, 모델 응답 내용에는 영향을 미치지 않습니다.

- instructions: 시스템 레벨의 지시어로, 해당 어시스턴트가 모든 대화에서 따르게 될 기본 규칙이나 역할을 정의합니다. 여기서는 "친절한 도움말 어시스턴트"라는 성격을 부여했습니다. (이 지시어는 기존 ChatGPT의 시스템 메시지와 유사한 역할입니다.)

- model: 어시스턴트가 사용할 언어 모델을 지정합니다. 최신 기능을 쓰려면 OpenAI가 Nov 2023 이후 출시한 모델(-1106가 붙은 모델명)을 권장합니다. 

- assistant 생성을 제공하는 모델
    GPT-4 계열:
    gpt-4 - 기본 GPT-4 모델
    gpt-4-turbo - 더 빠르고 효율적인 GPT-4
    gpt-4o - 최신 멀티모달 모델
    gpt-4o-mini - 경량화된 GPT-4o

    GPT-3.5 계열:
    gpt-3.5-turbo - 빠르고 효율적인 모델(추천)

    기타:
    dall-e-3 - 이미지 생성용
    whisper-1 - 음성 인식용
    tts-1, tts-1-hd - 텍스트 음성 변환용


- tools: 어시스턴트에 활성화할 도구 목록입니다. 기본적인 Q&A 챗봇에서는 특별한 툴이 필요 없으므로 비워두었습니다. (tools=[] 또는 생략) 나중에 코드 인터프리터나 함수 호출 등을 사용하게 될 경우 이 필드를 설정합니다.

어시스턴트가 성공적으로 생성되면 고유 ID (assistant.id)가 반환됩니다. 이 ID는 이후 대화 스레드에서 어떤 어시스턴트를 사용할지 지정할 때 필요하므로 저장해둡니다. (참고로, client.beta.assistants.list()를 호출하면 계정 내 모든 어시스턴트 목록을 확인할 수 있습니다.) 

이제 이 어시스턴트와 대화를 시작해보겠습니다. 대화를 하기 위해서는 **스레드(Thread)**를 생성해야 합니다. 스레드는 개별적인 대화 세션을 나타내며, 사용자와 어시스턴트 간의 메시지(Message) 목록을 포함합니다. 하나의 어시스턴트는 여러 개의 스레드를 가질 수 있는데, 예를 들어 같은 어시스턴트를 여러 사용자가 동시에 이용하면 각 사용자별로 별도 스레드가 생깁니다. 

스레드를 만들고, 그 안에 사용자 메시지를 추가한 뒤, 어시스턴트의 답변을 요청하는 일련의 과정을 코드로 수행해봅시다:


In [15]:
# 3. 새로운 대화 thread 생성 : 기억을 담당
thread = client.beta.threads.create()
# thread
# 4. 스레드에 사용자 메세지 추가
# user_message = 
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="서울 날씨 몇도까지 떨어져요?"
)
# user_message
# 5. 어시스턴트에게 답변 생성 요청 (과금)
# run = 
client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)
#run

Run(id='run_PxFPbtni1seEIIVjPWB4Eqbj', assistant_id='asst_4gBqxGrrNXDNLQiMg9XWT175', cancelled_at=None, completed_at=1765507817, created_at=1765507814, expires_at=None, failed_at=None, incomplete_details=None, instructions='당신은 유능하고 친절한 도움말 어시스턴트입니다. 사용자 질문에 20자이내로 답변하세요', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1765507816, status='completed', thread_id='thread_pTXrND1lYj73DnpNK5NxEq5i', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=14, prompt_tokens=69, total_tokens=83, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [18]:
# 6. OpenAI API 요청 후, 스레드의 모든 메세지 불러오기
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [39]:
import time
# messages.data[0]
reply = messages.data[0].content[0].text.value
role = messages.data[0].role
replyDate = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(messages.data[0].created_at))
print(f"{role}({replyDate}) : {reply}")

assistant(2025-12-12 11:50:17) : 최저 기온은 대략 0도입니다.


In [40]:
# 6-1 메세지를 시간 순서대로 정렬
messages.data

[Message(id='msg_DPNw0aNprB9BjkeZ0WRiNCOZ', assistant_id='asst_4gBqxGrrNXDNLQiMg9XWT175', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='최저 기온은 대략 0도입니다.'), type='text')], created_at=1765507817, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_PxFPbtni1seEIIVjPWB4Eqbj', status=None, thread_id='thread_pTXrND1lYj73DnpNK5NxEq5i'),
 Message(id='msg_qc0wVnLNImk8btZHwqsEoKp1', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='서울 날씨 몇도까지 떨어져요?'), type='text')], created_at=1765507812, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_pTXrND1lYj73DnpNK5NxEq5i')]

In [42]:
# 스칼라 데이터 리스트 sort
students = ['김', '홍', '박']
students.sort()
students

['김', '박', '홍']

In [43]:
time.time() # 1970.1.1 ~ 지금까지 초수

1765509357.7477725

In [49]:
fun = lambda student : student['score']
fun({'name':'홍', 'score':90})

90

In [52]:
# 딕셔너리 리스트 sort
students = [
    {'name':'홍', 'create_at':time.time()+60*60*24, "score":60},
    {'name':'김', 'create_at':time.time()-60*60*24, "score":90},
    {'name':'박', 'create_at':time.time(), "score":20},
]
sorted_students = sorted(students,
                        key=lambda student : student['create_at'],
                        #reverse=True
                        )
sorted_students

[{'name': '김', 'create_at': 1765423321.859713, 'score': 90},
 {'name': '박', 'create_at': 1765509721.859713, 'score': 20},
 {'name': '홍', 'create_at': 1765596121.859713, 'score': 60}]

In [71]:
# 6-1 메세지를 시간 순서대로 정렬
sorted_messages = sorted(messages.data,
                    key=lambda msg : msg.created_at
)
# 7. 답변들 history
for msg in sorted_messages:
    #print(msg)
    content = msg.content[0].text.value
    role = msg.role
    dateStr = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(msg.created_at))
    print("{} ({}) : {}".format(role, dateStr, content))

user (2025-12-12 11:50:12) : 서울 날씨 몇도까지 떨어져요?
assistant (2025-12-12 11:50:17) : 최저 기온은 대략 0도입니다.


위 코드 설명:

- client.beta.threads.create(): 새로운 대화 스레드를 생성합니다. 반환된 thread 객체는 고유 ID (thread.id)를 갖습니다.

- client.beta.threads.messages.create(...): 특정 스레드에 새로운 메시지를 추가합니다. role 파라미터로 이 메시지의 주체를 지정하는데, 사용자 메시지이므로 "user"로 설정하고 content에 사용자의 질문 내용을 넣습니다.

- client.beta.threads.runs.create_and_poll(...): 해당 스레드를 주어진 어시스턴트로 실행(run)합니다. assistant_id를 우리가 생성한 어시스턴트의 ID로 지정해야 합니다. 여기서는 create_and_poll를 사용하여 실행이 완료될 때까지 블록합니다. (이 함수는 내부적으로 주기적으로 run.status를 체크하여 완료될 때까지 기다립니다.)

    >참고: create_and_poll를 사용하지 않고 runs.create를 쓰는 경우, 즉시 run 객체를 받고 수동으로 상태를 확인해야 합니다. 간단한 예에서는 편의를 위해 create_and_poll로 한 번에 처리했습니다.

- 대화 실행이 완료되면 (run.status == "completed") 해당 스레드에는 사용자 질문과 어시스턴트 답변, 총 두 개의 메시지가 들어있게 됩니다. 이를 threads.messages.list로 가져와서 출력했습니다.

어시스턴트가 우리의 지시어에 따라 친절한 톤으로 답변한 것을 볼 수 있습니다. 이처럼 스레드 생성 -> 메시지 추가 -> 실행 -> 결과 조회의 흐름으로 어시스턴트와의 기본 대화를 구현할 수 있습니다.

>🔍 참고: thread는 대화의 흐름(메시지 목록)을 나타내고, run은 어시스턴트가 해당 스레드의 마지막 사용자 메시지에 응답하는 실행 과정을 가리킵니다. 일반적으로 하나의 사용자 메시지에 대해 하나의 run이 생성되며, 어시스턴트가 답변을 완성하면 run.status가 "completed"로 바뀝니다. (코드에서는 create_and_poll를 사용해 이 과정을 추상화했습니다.)

이제 기본적인 Q&A 챗봇 형태의 어시스턴트를 만들고, 질문에 답변을 받는 방법을 확인했습니다. 다음으로는 파일을 어시스턴트에 활용하는 방법과 코드 실행 도구를 사용하는 방법을 살펴보겠습니다.


In [2]:
# assistant chatbot
# 1. 어시스턴트 생성 (id확보)
# 2. 스레드 생성(id확보) - 새대화 시작하고 지금부터 대화 history 저장
# while True:
    # 3. 스레드에 사용자 질문 메세지 추가
    # 4. run 실행(어시스턴트가 메세지를 읽고 응답 - 과금)
    # 5. 응답 메세지를 스레드에서 추출하여 출력
# 6. 대화 이력 history.txt로 파일 저장
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import time
import os
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
# client 생성
load_dotenv()
client = OpenAI()
# 1. assistant 생성
assistant_cs = client.beta.assistants.create(
    name="CustomerQnABot",
    instructions='당신은 고객 지원 쳇봇입니다. 사용자 문의에 대해 간략히 답변해 주세요',
    model = 'gpt-4o-mini'
)
# 2. thread 생성 : 대화 시작
thread_cs = client.beta.threads.create()
print("어서오세요(exit를 입력하시면 종료됩니다)")
while True:
    user_input = input("User :").strip()
    if user_input.lower() in ['exit', '종료']:
        print("이용해 주셔서 감사합니다. 다음에 뵙겠습니다.")
        break
    if user_input == "":
        continue
    # 3. 스레드에 사용자 질문 메세지 추가
    client.beta.threads.messages.create(
        thread_id=thread_cs.id,
        role="user",
        content=user_input
    )
    # 4. run 실행(어시스턴트가 메세지를 읽고 응답 - 과금)
    client.beta.threads.runs.create_and_poll(
        thread_id=thread_cs.id,
        assistant_id=assistant_cs.id
    )
    # 5. 응답 메세지를 스레드에서 추출하여 출력
    messages = client.beta.threads.messages.list(thread_id=thread_cs.id)
    reply = messages.data[0].content[0].text.value # 최신답변
    print("Assistant :", reply)

어서오세요(exit를 입력하시면 종료됩니다)
User :2020년 월드시리즈 우승팀은?
User :오케이
User :종료
이용해 주셔서 감사합니다. 다음에 뵙겠습니다.
이상의 대화 이력을 파일에 백업합니다


In [ ]:
# 6. 대화 이력 data/ch7_history.txt로 파일 저장
print('이상의 대화 이력을 파일에 백업합니다')
sorted_messages = sorted(messages.data, 
                        key=lambda msg : msg.created_at)
with open('data/ch7_history.txt', 'w', encoding='utf-8') as f:
    for msg in sorted_messages:
        # role / content / dateStr
        role = msg.role
        content = 